In [ ]:
import numpy as np
import dpdata

1.**计算红外光谱**：首先需要计算偶极矩-偶极矩时间关联函数。下面展示了计算时间关联函数的代码，其中dt对应了分子动力学模拟的步长，这里为0.0005 ps；window对应每个时刻下时间关联函数的乘积项数目，这里为2000，dt $\cdot$ window的值1 ps即为之前提到的时间窗口的值；$r_c$代表截断半径，即考虑每个水分子周围多少范围内的水分子与其相互作用，对于纯水体系一般设为6 Å，也即考虑每个水分子周围两层水对其振动光谱的影响，超过6 Å范围的水分子对其振动光谱影响很小。

In [ ]:
from md_spectra import compute_and_plot_ir

dt = 0.0005
window = 50000
width = 25
temperature = 330.0

atomic_dipole = np.load('atomic_dipole_wan.npy')
atomic_dipole = atomic_dipole.reshape(atomic_dipole.shape[0], -1, 3)

compute_and_plot_ir(
    atomic_dipole=atomic_dipole,
    dt=dt,
    window=window,
    width=width,
    temperature=temperature,
    plot=True,
    save_plot=True,
    plot_path='ir.png',
    save_data=True,
    data_path='ir.dat'
)

**计算拉曼光谱**  
本单元格展示如何调用 `compute_raman_spectra` 计算和绘制拉曼光谱，包括各向同性和各向异性部分。`width_iso` 和 `width_aniso` 分别控制不同类型谱线的宽度。结果会保存为图片和数据文件。

In [ ]:
from md_spectra import compute_raman_spectra

window = 50000
width_iso = 25
width_aniso = 240
temperature = 330.0

atomic_polar = np.load('atomic_polar_wan.npy')
atomic_polar = atomic_polar.reshape(atomic_polar.shape[0], -1, 3, 3)

compute_raman_spectra(
    atomic_polar=atomic_polar,
    dt=dt,
    window=window,
    width_iso=width_iso,
    width_aniso=width_aniso,
    temperature=temperature,
    plot=True,
    save_plot=True,
    plot_paths={
        'iso': 'raman_iso.png',
        'aniso': 'raman_aniso.png',
        'aniso_low': 'raman_aniso_low.png'
    },
    save_data=True,
    data_paths={
        'iso': 'raman_iso.dat',
        'aniso': 'raman_aniso.dat',
        'aniso_low': 'raman_aniso_low.dat'
    }
)

**计算轨迹中的原子偶极矩**  
本单元格用于从分子动力学轨迹和 Wannier 中心数据中计算每个水分子的原子偶极矩。通过 `compute_atomic_dipole_from_traj` 函数处理原始数据，并保存为 numpy 文件，便于后续分析。

In [ ]:
from md_spectra import compute_atomic_dipole_from_traj

a0 = 0.52917721067

traj = dpdata.System("traj", fmt="deepmd/npy")
wannier: np.ndarray = np.loadtxt('wannier_dipole.raw')
# reshape if needed
if wannier.ndim == 2:
    wannier = wannier.reshape(traj.get_nframes(), -1, 3)

atomic_dipole = compute_atomic_dipole_from_traj(
    traj=traj,
    wannier=wannier,
    type_O=1,
    type_H=2,
    r_bond=1.3,
    a0=a0,
    save=True,
    save_path='atomic_dipole_wan.npy',
    print_info=True
)

**提取轨迹中的原子极化率张量**  
本单元格演示如何从轨迹和 Wannier 极化率原始数据中提取每个水分子的原子极化率张量。调用 `extract_atomic_polar_from_traj` 并保存结果，为后续拉曼和 SFG 分析做准备。

In [ ]:
from md_spectra import extract_atomic_polar_from_traj

traj = dpdata.System("traj", fmt="deepmd/npy")
polar: np.ndarray = np.loadtxt('wannier_polar.raw')
polar = -polar.reshape(polar.shape[0], -1, 3, 3)
# 调用转写后的函数
atomic_polar = extract_atomic_polar_from_traj(
    traj=traj,
    polar=polar,
    type_O=1,
    type_H=2,
    r_bond=1.3,
    save=True,
    save_path='atomic_polar_wan.npy',
    print_info=True
)

**计算表面红外光谱**  
本单元格展示如何调用 `compute_surface_ir_spectra` 计算不同 z 区域（水的表面和体相）下的红外光谱。可视化和保存表面与体相的 IR 光谱，便于分析界面效应。

In [ ]:
from md_spectra import compute_surface_ir_spectra

dt = 0.0005
window = 50000

h2o = np.load("h2o.npy")
atomic_dipole = np.load('atomic_dipole_wan.npy')
atomic_dipole = atomic_dipole.reshape(atomic_dipole.shape[0], -1, 3)

compute_surface_ir_spectra(
    h2o=h2o,
    atomic_dipole=atomic_dipole,
    dt=dt,
    window=window,
    z1_min=16.0,
    z1_max=17.4,
    z2_min=20.0,
    z2_max=25.0,
    z3_min=27.6,
    z3_max=29.0,
    z_total_min=16.0,
    z_total_max=29.0,
    z_bin=0.4,
    width=25,
    temperature=330.0,
    plot=True,
    save_plot=True,
    plot_path='ir_sp.png',
    save_data=True,
    data_path='ir_sp.dat'
)

**计算表面拉曼光谱**  
本单元格用于调用 `compute_surface_raman_spectra`，分别计算水的表面和体相的拉曼光谱，包括各向同性、各向异性和低频部分，并保存结果。

In [ ]:
from md_spectra import compute_surface_raman_spectra

dt = 0.0005
window = 50000

h2o = np.load("h2o.npy")
atomic_polar = np.load('atomic_polar_wan.npy')
atomic_polar = atomic_polar.reshape(atomic_polar.shape[0], -1, 3, 3)

compute_surface_raman_spectra(
    h2o=h2o,
    atomic_polar=atomic_polar,
    dt=dt,
    window=window,
    z_total_min=16.0,
    z_total_max=29.0,
    z1_min=16.0,
    z1_max=17.4,
    z2_min=20.0,
    z2_max=25.0,
    z3_min=27.6,
    z3_max=29.0,
    z_bin=0.4,
    width=25,
    temperature=330.0,
    plot=True,
    save_plot=True,
    plot_paths={
        'iso': 'raman_iso.png',
        'aniso': 'raman_aniso.png',
        'aniso_low': 'raman_aniso_low.png'
    },
    save_data=True,
    data_paths={
        'iso': 'raman_iso.dat',
        'aniso': 'raman_aniso.dat',
        'aniso_low': 'raman_aniso_low.dat'
    }
)

**计算表面和界面 SFG 光谱**  
本单元格演示如何调用 `compute_surface_sfg` 计算水的表面/界面 SFG（和频生成）光谱。通过设置不同的空间参数，可以分析界面分子的振动响应，并输出图像和数据。

In [ ]:
from md_spectra import compute_surface_sfg

dt = 0.0005
window = 50000

h2o = np.load("h2o.npy")
atomic_dipole = np.load('atomic_dipole_wan.npy').reshape(h2o.shape[0], -1, 3)
atomic_polar = np.load('atomic_polar_wan.npy').reshape(h2o.shape[0], -1, 3, 3)

compute_surface_sfg(
    h2o=h2o,
    atomic_dipole=atomic_dipole,
    atomic_polar=atomic_polar,
    dt=dt,
    window=window,
    z0=22.5,
    zc=2.5,
    zw=2.6,
    rc=6.75,
    width=50,
    temperature=330.0,
    plot=True,
    save_plot=True,
    plot_path='sfg.png',
    save_data=True,
    data_path='SFG.dat'
)

**Wannier 文件处理与原子坐标/盒子生成**  
本单元格用于收集并合并指定目录下的 Wannier 文件，生成标准化的原子坐标（coord.npy）和盒子信息（box.npy），并通过 `set_cells` 和 `set_lumped_wfc` 设置结构的盒子参数及 Wannier 中心的归并信息。为后续偶极矩和极化率等分析提供标准化输入数据。

In [ ]:
from ase.io import read
from md_spectra import set_cells, set_lumped_wfc

# collect the wannier file from workdir and concatenate these files 
def concatenate_files(root_dir, wc_filename, output_file):
    with open(output_file, 'w') as outfile:
        for dirpath, dirnames, filenames in os.walk(root_dir):
            for file in filenames:
                if file == wc_filename:
                    with open(os.path.join(dirpath, file), 'r') as infile:
                        outfile.write(infile.read())

# convert the coordinates file to npy form and generate a box file of npy form 
def convert_npy(stc_list_file, a, b, c):
    stc_list = read(stc_list_file, index=":")
    atom_pos = []
    box = np.zeros((len(stc_list), 9))
    box[:, 0] = a
    box[:, 4] = b
    box[:, 8] = c

    for stc in stc_list:
        for atom in stc:
            if atom.symbol == 'O':
                atom_pos.append(atom.position)
            elif atom.symbol == 'H':
                atom_pos.append(atom.position)

    convert_coord = np.reshape(atom_pos, (len(stc_list), -1))
    np.save("coord.npy", convert_coord)
    np.save("box.npy", box)

concatenate_files('/public/home/xldu/ai2-kit/water/workdir/run-02', '64water_wannier.xyz', 'wannier.xyz')
convert_npy('wannier.xyz', 12.42, 12.42, 12.42)

lumped_dict = {"O": 4}
cell = [12.42, 12.42, 12.42]
stc_list_file = 'wannier.xyz'
stc_list = read(stc_list_file, index=":")
stc_list = set_cells(stc_list, cell)
set_lumped_wfc(stc_list, lumped_dict)
# lumped_dict
# key: element symbol, the atoms of which are the relative atoms of wannier centers
# value: expected neighbor wannier center number to be lumped.
# e.g. for water, the oxygen has 4 neighbor wannier centers, so the lumped_dict = {"O": 4}